# Data Cleaning

## Introduction

This notebook goes through a necessary step of any data science project - data cleaning. Data cleaning is a time consuming and unenjoyable task, yet it's a very important one. Keep in mind, "garbage in, garbage out". Feeding dirty data into a model will give us results that are meaningless.

Specifically, we'll be walking through:

1. **Getting the data -** in this case, we'll be scraping data from a website
2. **Cleaning the data -** we will walk through popular text pre-processing techniques
3. **Organizing the data -** we will organize the cleaned data into a way that is easy to input into other algorithms

The output of this notebook will be clean, organized data in two standard text formats:

1. **Corpus** - a collection of text
2. **Document-Term Matrix** - word counts in matrix format

## Problem Statement

As a reminder, our goal is to look at transcripts of Game of Thrones and note the similarities and differences between the different characters.

## Getting the data

Luckily, there are wonderful people online that keep track Game of Thrones transcripts. [Genius](https://genius.com/artists/Game-of-thrones) is one such site.

In [1]:
# Web scraping, pickle imports
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup
import pickle
import os

In [1]:
seasons_url = ['https://genius.com/albums/Game-of-thrones/Season-1-scripts',
               'https://genius.com/albums/Game-of-thrones/Season-2-scripts',
               'https://genius.com/albums/Game-of-thrones/Season-3-scripts',
               'https://genius.com/albums/Game-of-thrones/Season-4-scripts', 
               'https://genius.com/albums/Game-of-thrones/Season-5-scripts', 
               'https://genius.com/albums/Game-of-thrones/Season-6-scripts', 
               'https://genius.com/albums/Game-of-thrones/Season-7-scripts', 
               'https://genius.com/albums/Game-of-thrones/Season-8-scripts']

In [ ]:
def get_episode_link(seasons_url):
    episodes_link = []
    for link in seasons_url:
        req = Request(link, headers={'User-Agent': 'Mozilla/5.0'})
        uclient = urlopen(req)
        page_html = uclient.read()
        uclient.close()
        page_soup = soup (page_html, "html.parser")
        div_tag=page_soup.findAll('div', attrs={'class':'chart_row-content'})
        for title in div_tag:
            link = title.find('a')['href']
            episodes_link.append(link)
    return episodes_link
        
episodes_link = get_episode_link(seasons_url)

In [ ]:
def get_transcript(episodes_link):
    transcripts = []
    for link in episodes_link:
        req = Request(link, headers={'User-Agent': 'Mozilla/5.0'})
        uclient = urlopen(req)
        page_html = uclient.read()
        uclient.close()
        page_soup = soup (page_html, "html.parser")
        div_tag=page_soup.find('div', attrs={'class':'song_body-lyrics'})
        p_tag = div_tag.findAll('p')
        episode_transcript = []
        for para in p_tag:
            text = para.text
            episode_transcript.append(text)
        episode_transcript = " ".join(episode_transcript)
        name = link.split('/')[-1]
        file = open("Scripts/" + name + ".txt","w", encoding="utf-8") 
        file.write(episode_transcript)
        transcripts.append(episode_transcript)    
    return transcripts

transcripts = get_transcript(episodes_link)

In [ ]:
def remove_strings(string):
    if len(string) == 0:
        return False
    if ':' not in string:
        return False
    if 'CUT TO:' in string:
        return False
    return True

dict_speeches = []
for file in os.listdir('Scripts/'):
    with open('Scripts/' + file, "r", encoding='utf-8') as f:
        text = f.read()
        text = text.split('\n')
        text = list(filter(remove_strings, text))
        for dialogue in text:
            dict_speeches.append(dialogue)

In [ ]:
data = {}
for speech in dict_speeches:
    speaker = speech.split(":")[0]
    dialogue = speech.split(":")[1]
    if speaker not in speeches:
        data[speaker] = dialogue
    else:
        data[speaker] += dialogue

In [ ]:
data.keys()

In [ ]:
with open('got.pkl', 'wb') as f:
    pickle.dump(data, f)

In [68]:
with open('got.pkl', 'rb') as f:
    data = pickle.load(f)

In [69]:
# More checks
data['Eddard Stark']

' Your pardon, your Grace.  She did nothing I did not command.  I am the King’s Hand -  - trying to keep the peace - He butchered my men.  Jaime has fled the city. Give me leave to bring him back to justice.  If we don’t act… there will be a war.  And what about Jaime Lannister? What about Jaime?  As you command your Grace. With your leave, I will return to Winterfell and set matters straight. The hunt? The Targaryen girl -  These men, were they flying a sigil? … A banner.  You’re describing Ser Gregor Clegane.  I cannot give you back your homes or restore your dead to life. But perhaps I can give you justice in the name of our king, Robert. Lord Beric Dondarrion.  You shall have the command. Assemble 100 men and ride to Ser Gregor’s keep.  In the name of Robert of the House Baratheon, the First of his Name, King of the Andals and the First Men, Lord of the Seven Kingdoms and Protector of the Realm, I charge you to bring the king’s justice to the false knight Gregor Clegane and all tho

## Cleaning Data

When dealing with numerical data, data cleaning often involves removing null values and duplicate data, dealing with outliers, etc. With text data, there are some common data cleaning techniques, which are also known as text pre-processing techniques.

With text data, this cleaning process can go on forever. There's always an exception to every cleaning step. So, we're going to follow the MVP (minimum viable product) approach - start simple and iterate. Here are a bunch of things you can do to clean your data. We're going to execute just the common cleaning steps here and the rest can be done at a later point to improve our results.

**Common data cleaning steps on all text:**
* Make text all lower case
* Remove punctuation
* Remove numerical values
* Remove common non-sensical text (/n)
* Tokenize text
* Remove stop words

**More data cleaning steps after tokenization:**
* Stemming / lemmatization
* Parts of speech tagging
* Create bi-grams or tri-grams
* Deal with typos
* And more...

In [70]:
# Let's take a look at our data again
data.keys()

dict_keys(['Eddard Stark', 'Cersei Baratheon', 'Robert Baratheon', 'Irri', 'Daenerys Targaryen', 'Robb Stark', 'Bran Stark', 'Theon Greyjoy', 'Osha', 'Wildling', 'Wildling 2', 'Tyrion Lannister', 'Mord', 'Arya Stark', 'Syrio Forel', 'Viserys Targaryen', 'Jorah Mormont', 'Jora', 'Lysa Arryn', 'Robin Arryn', 'Catelyn Stark', 'Knight 1', 'Knight 2', 'Knight 3', 'Knight 4', 'Knight 5', 'Ser Vardis', 'Bronn', 'Lancel Lannister', 'Renly Baratheon', 'Barriston', 'Barristan Selmy', 'Man', 'Grand Maester Pycelle', 'Petyr Baelish', 'Beric Dondarrion', 'Mordane', 'Sansa Stark', 'Joffrey Baratheon', 'Ros', 'Khal Drogo', 'DAENERYS TARGARYEN', 'DAENERYS', 'JAIME', 'TYRION', 'SANSA', 'BRAN', 'BRIENNE', 'JON', 'GENDRY', 'ARYA', 'BOTH', 'JORAH', 'ROYCE', 'WOLKAN', 'THEON', 'MAN', 'DAVOS', 'GILLY', 'WOMAN', 'TEELA', 'TORMUND', 'EDD', 'BERIC', 'JON (V.O.)', 'JON(V.O.)', 'SAMWELL', 'MISSANDEI', 'GREY WORM', 'PODRICK', 'SANDOR', 'JORAH (O.S.)', 'LYANNA (O.S.)', 'LYANNA', 'LYANN', 'PODRICK (V.O.)', 'EPISODE

In [71]:
# We can either keep it in dictionary format or put it into a pandas dataframe
import pandas as pd
pd.set_option('max_colwidth',150)

data_df = pd.DataFrame.from_dict([data]).transpose().reset_index()
data_df.columns = ['speaker','transcript']
data_df

,speaker,transcript
0,"""Robb, I write to you with a heavy heart. Our good king Robert is dead, killed from wounds he took in a boar hunt. Father has been charged with tr...","come to King's Landing, swear fealty to King Joffrey and prevent any strife between the great houses of Lannister and Stark. Your faithfull siste..."
1,"""STEELSHANKS"" WALTON",Why? Absolutely not. I've got orders from Lord Bolton. To deliver you to your father at King's Landing. I serve Lord Bolton. Any appreciation yo...
2,A VOICE,"It’s Maester Luwin, my lord."
3,AEMON,"And she's alone, under siege, no family to guide her or protect her. Her last relation thousands of miles away, useless, dying. A Targaryen, alon..."
4,AEMON (panicked),Egg! Egg.. I dreamed... that I was old
5,AERON,"Lord, take your servant Balon back beneath the waves. Feed the creatures of your kingdom on his flesh. Pull his bones down to your depths to rest..."
6,AERSON,Listen to the waves. Listen to the god. He is speaking to us and he says we shall have no king but Euron Greyjoy. Let the sea wash your follies a...
7,AHSA,"There are other ways to serve, my prince. What all free people want. My freedom."
8,ALL,"And now his watch has ended. We do! We do! Save our sons from war, we pray. Halfman! Halfman! Halfman! Hail King Joffrey. Hail Queen Margaery. St..."
9,ALL THREE,(Theon joins a little late). What's dead may never die.


In [72]:
data_df = data_df[data_df['speaker'].apply(lambda x: len(x) <= 50)]
s = data_df.transcript.str.len().sort_values(ascending=False).index
data_df = data_df.reindex(s)
data_df = data_df[:15]
data_df

,speaker,transcript
855,TYRION,"Your Grace, I know my brother- He came here alone, knowing full well how he'd be received. Why would he do that if he weren't telling the truth? ..."
117,CERSEI,"Your mother might have prepared you. You flowered, my dear. No more. In what way? Wait until you birth a child. You're a woman now. Do you have a..."
341,JAIME,"She did. She lied to me as well. She never had any intention of sending her army north. She has Euron Greyjoy's fleet and 20,000 fresh troops. Th..."
156,DAENERYS,Who stabbed him in the back and cut his throat. Who sat down on the Iron Throne and watched as his blood poured onto the floor. He told me other ...
358,JON,We need every man we can get. I thought we'd lost you. How did you find each other? The Umbers? How long do we have? We can't beat them in a stra...
746,SANSA,"You're right. We can't trust him. He attacked my father in the streets. He tried to destroy my house and my family, the same as he did yours. You..."
176,DAVOS,"You are now. Look, I made it through most of my years without ever getting near a fight, but then I survived the Battle of the Bastards. Right ou..."
884,VARYS,"Oh, Lord Tyrion. Come in, come in. Oh, always in confidence. Oh, no. It's a wonderful time. End your life? Proof? Will there be a trial? I have n..."
29,ARYA,You make my weapon yet? You should make mine first. And make sure it's stronger than this. Are you going to be down in the crypt? But you're a fi...
866,TYWIN,Wolfsbane. A rare substance. This is no common assassin. I don't care if you hanged a hundred. A man tried to kill me. I want his name and I want...


In [73]:
# Apply text cleaning techniques
import re
import string

def clean_text_round(text):
    '''Make text lowercase, remove punctuation, remove words containing numbers and non-sensical text line \n.'''
    text = text.lower()
    text = re.sub('\n', ' ', text)
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)

    return text

clean_text = lambda x: clean_text_round(x)

In [74]:
# Let's take a look at the updated text
data_df['transcript'] = pd.DataFrame(data_df.transcript.apply(clean_text))
data_df['speaker'] = pd.DataFrame(data_df.speaker.apply(clean_text))

In [75]:
data_df

,speaker,transcript
855,tyrion,your grace i know my brother he came here alone knowing full well how hed be received why would he do that if he werent telling the truth i was a...
117,cersei,your mother might have prepared you you flowered my dear no more in what way wait until you birth a child youre a woman now do you have any idea ...
341,jaime,she did she lied to me as well she never had any intention of sending her army north she has euron greyjoys fleet and fresh troops the golden co...
156,daenerys,who stabbed him in the back and cut his throat who sat down on the iron throne and watched as his blood poured onto the floor he told me other st...
358,jon,we need every man we can get i thought wed lost you how did you find each other the umbers how long do we have we cant beat them in a straight fi...
746,sansa,youre right we cant trust him he attacked my father in the streets he tried to destroy my house and my family the same as he did yours you vouch ...
176,davos,you are now look i made it through most of my years without ever getting near a fight but then i survived the battle of the bastards right outsid...
884,varys,oh lord tyrion come in come in oh always in confidence oh no its a wonderful time end your life proof will there be a trial i have no proof only ...
29,arya,you make my weapon yet you should make mine first and make sure its stronger than this are you going to be down in the crypt but youre a fighter ...
866,tywin,wolfsbane a rare substance this is no common assassin i dont care if you hanged a hundred a man tried to kill me i want his name and i want his h...


In [76]:
# Let's take a look at the transcript for Arya Stark
data_df[data_df['speaker']=='arya']['transcript']

29     you make my weapon yet you should make mine first and make sure its stronger than this are you going to be down in the crypt but youre a fighter ...
Name: transcript, dtype: object

**NOTE:** This data cleaning aka text pre-processing step could go on for a while, but we are going to stop for now. After going through some analysis techniques, if you see that the results don't make sense or could be improved, you can come back and make more edits such as:
* Mark 'cheering' and 'cheer' as the same word (stemming / lemmatization)
* Combine 'thank you' into one term (bi-grams)
* And a lot more...

## Organizing the data

I mentioned earlier that the output of this notebook will be clean, organized data in two standard text formats:
1. **Corpus -** a collection of text
2. **Document-Term Matrix -** word counts in matrix format

### Corpus

We already created a corpus in an earlier step. The definition of a corpus is a collection of texts, and they are all put together neatly in a pandas dataframe here.

In [77]:
# Let's take a look at our dataframe
data_df

,speaker,transcript
855,tyrion,your grace i know my brother he came here alone knowing full well how hed be received why would he do that if he werent telling the truth i was a...
117,cersei,your mother might have prepared you you flowered my dear no more in what way wait until you birth a child youre a woman now do you have any idea ...
341,jaime,she did she lied to me as well she never had any intention of sending her army north she has euron greyjoys fleet and fresh troops the golden co...
156,daenerys,who stabbed him in the back and cut his throat who sat down on the iron throne and watched as his blood poured onto the floor he told me other st...
358,jon,we need every man we can get i thought wed lost you how did you find each other the umbers how long do we have we cant beat them in a straight fi...
746,sansa,youre right we cant trust him he attacked my father in the streets he tried to destroy my house and my family the same as he did yours you vouch ...
176,davos,you are now look i made it through most of my years without ever getting near a fight but then i survived the battle of the bastards right outsid...
884,varys,oh lord tyrion come in come in oh always in confidence oh no its a wonderful time end your life proof will there be a trial i have no proof only ...
29,arya,you make my weapon yet you should make mine first and make sure its stronger than this are you going to be down in the crypt but youre a fighter ...
866,tywin,wolfsbane a rare substance this is no common assassin i dont care if you hanged a hundred a man tried to kill me i want his name and i want his h...


In [78]:
# Let's pickle it for later use
data_df.to_pickle("corpus.pkl")

### Document-Term Matrix

For many of the techniques we'll be using in future notebooks, the text must be tokenized, meaning broken down into smaller pieces. The most common tokenization technique is to break down text into words. We can do this using scikit-learn's CountVectorizer, where every row will represent a different document and every column will represent a different word.

In addition, with CountVectorizer, we can remove stop words. Stop words are common words that add no additional meaning to text such as 'a', 'the', etc.

In [79]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_df.transcript)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_df['speaker']
data_dtm

,abandon,abandoned,abandoning,abate,abduct,abilities,ability,able,abolishing,aboveaverage,...,youngest,youre,yourewell,youve,youyou,yunkai,zealot,zo,zuriff,ēza
speaker,,,,,,,,,,,,,,,,,,,,,
tyrion,1,2,0,0,1,1,0,6,2,1,...,2,69,0,27,0,2,0,0,0,0
cersei,1,1,0,0,0,0,0,0,0,0,...,1,38,0,15,0,0,0,0,0,0
jaime,2,0,1,0,0,0,0,5,0,0,...,0,44,0,9,1,0,0,0,0,0
daenerys,0,2,0,0,0,0,0,3,0,0,...,0,17,0,3,0,8,0,3,0,1
jon,1,0,0,0,0,0,0,0,0,0,...,0,34,1,6,0,0,0,0,0,0
sansa,2,0,3,0,0,0,0,2,0,0,...,0,31,0,6,0,0,0,0,2,0
davos,1,0,0,0,0,0,1,0,0,0,...,0,20,0,2,0,0,0,0,0,0
varys,1,2,0,0,0,0,0,2,0,0,...,0,7,0,4,0,2,0,0,0,0
arya,0,0,0,0,0,0,0,1,0,0,...,1,30,0,5,0,0,0,0,0,0


In [80]:
# Let's pickle it for later use
data_dtm.to_pickle("dtm.pkl")